# CVaR portfolio optimization benchmarks
This notebook uses Fortitudo Technologies' Investment Analysis module to compute CVaR portfolio optimization results for the benchmark problems from https://github.com/fortitudo-tech/cvar-optimization-benchmarks

The initial market data uses a lognormal P&L simulation, similar to the one that follows with the open-source fortitudo.tech Python package: https://github.com/fortitudo-tech/fortitudo.tech

The problems become increasingly hard, ranging from CVaR efficient frontier optimization of long-only cash portfolios without transaction costs to CVaR optimization of derivatives portfolios with transaction costs, absolute leverage constraints, and risk budgets.

Finally, we add non-uniform scenario probabilities to all these problems through a Sequential Entropy Pooling CVaR stress-test (see Chapter 5 in the Portfolio Construction and Risk Management book: https://antonvorobets.substack.com/p/pcrm-book).

In total, there are 24 CVaR optimization problems that are optimized with 100 different expected return vectors. Note that for portfolios that contain derivatives Entropy Pooling is applied to compute consistent derivatives expected returns as described in: https://ssrn.com/abstract=4825945 

You can use the results to see which CVaR optimization problems are possible to solve and in what time. You can potentially compare the speed and stability to your own CVaR optimization implementations.

If you can solve all problems using a Python API significantly faster than in this Notebook, and your implementation is not publicly available, please contant us at software@fortitudo.tech. We might be willing to buy your implementation.

See the README file for more information on the CPU used to compute the results.

In [1]:
# Load standard Python packages
import numpy as np
from time import time

# Load the Investment Analysis module
from fortitudo.tech import investment_analysis as ia  # only available for paid subscribers

# Load P&L simulation and instrument information

In [2]:
instruments, pnl, p = ia.load_pnl(
    'data/instruments.csv', 'data/pnl.csv')  # full
instruments0, _, _ = ia.load_pnl(
    'data/instruments0.csv', 'data/pnl.csv')  # full without transaction costs
instruments_cash, pnl_cash, _ = ia.load_pnl(
    'data/instruments_cash.csv', 'data/pnl_cash.csv')  # cash only
instruments0_cash, _, _ = ia.load_pnl(
    'data/instruments0_cash.csv', 'data/pnl_cash.csv')  # only cash without transaction costs

In [3]:
stats_prior = ia.individual_stats(pnl, p, alpha=0.9, include_mean=True)
stats_prior

,Mean,Vol,Skew,Kurt,90%-VaR,90%-CVaR
"(0, DM Gov)",0.027,0.036,0.103,2.980,0.018,0.034
"(1, Corp IG)",0.035,0.061,0.169,2.955,0.043,0.069
"(2, Corp HY)",0.045,0.102,0.277,3.032,0.081,0.121
"(3, EM Gov)",0.047,0.101,0.275,3.030,0.079,0.118
"(4, DM Equities)",0.064,0.171,0.482,3.395,0.143,0.204
"(5, EM Equities)",0.081,0.222,0.600,3.515,0.185,0.257
"(6, Private Equity)",0.129,0.241,0.646,3.701,0.157,0.236
"(7, Infrastructure)",0.084,0.176,0.489,3.421,0.128,0.191
"(8, Real Estate)",0.065,0.132,0.374,3.264,0.098,0.147
"(9, Hedge Funds)",0.063,0.095,0.266,3.125,0.055,0.093


In [4]:
# Visualize instruments information including holding and proportional trading costs
instruments

,hold,buy,sell,mv / exp
DM Gov,0.0023,0.00050,0.00050,1.000000
Corp IG,0.0033,0.00100,0.00100,1.000000
Corp HY,0.0060,0.00200,0.00200,1.000000
EM Gov,0.0044,0.00200,0.00200,1.000000
DM Equities,0.0049,0.00100,0.00100,1.000000
EM Equities,0.0071,0.00150,0.00150,1.000000
Private Equity,0.0356,0.00300,0.00300,1.000000
Infrastructure,0.0152,0.00300,0.00300,1.000000
Real Estate,0.0133,0.00300,0.00300,1.000000
Hedge Funds,0.0171,0.00300,0.00300,1.000000


In [5]:
# Visualize instruments information including holding costs
instruments0

,hold,buy,sell,mv / exp
DM Gov,0.0023,0.0,0.0,1.000000
Corp IG,0.0033,0.0,0.0,1.000000
Corp HY,0.0060,0.0,0.0,1.000000
EM Gov,0.0044,0.0,0.0,1.000000
DM Equities,0.0049,0.0,0.0,1.000000
EM Equities,0.0071,0.0,0.0,1.000000
Private Equity,0.0356,0.0,0.0,1.000000
Infrastructure,0.0152,0.0,0.0,1.000000
Real Estate,0.0133,0.0,0.0,1.000000
Hedge Funds,0.0171,0.0,0.0,1.000000


In [6]:
# Visualize cash instruments information including holding costs
instruments0_cash

,hold,buy,sell,mv / exp
DM Gov,0.0023,0.0,0.0,1.0
Corp IG,0.0033,0.0,0.0,1.0
Corp HY,0.0060,0.0,0.0,1.0
EM Gov,0.0044,0.0,0.0,1.0
DM Equities,0.0049,0.0,0.0,1.0
EM Equities,0.0071,0.0,0.0,1.0
Private Equity,0.0356,0.0,0.0,1.0
Infrastructure,0.0152,0.0,0.0,1.0
Real Estate,0.0133,0.0,0.0,1.0
Hedge Funds,0.0171,0.0,0.0,1.0


# Stress test P&L assumptions with Sequential Entropy Pooling
Implement a 50% increase in the CVaR of DM Equities, see Chapter 5 in the Portfolio Construction and Risk Management book: https://antonvorobets.substack.com/p/pcrm-book

In [7]:
# Create Entropy Pooling object and add some views
ep = ia.EntropyPooling(pnl, p)
ep.add_view('90-CVaR', 4, '=', 1.5 * stats_prior['90%-CVaR'].iloc[4])  # (4, DM Equities)

# Compute posterior probabilities
q = ep.compute_posterior_probability(heuristic='H1')  # H1 Sequential Entropy Pooling algorithm
np.save('data/q.npy', q) # for posterior optimization

# Compute posterior stats and print
stats_posterior = ia.individual_stats(pnl, q, alpha=0.9, include_mean=True)
stats_posterior

Posterior probabilities computed in 0.07 seconds with effective number of scenarios 9161 / 10000 and relative entropy 0.0876.


,Mean,Vol,Skew,Kurt,90%-VaR,90%-CVaR
"(0, DM Gov)",0.028,0.036,0.100,2.963,0.018,0.034
"(1, Corp IG)",0.031,0.061,0.189,2.959,0.046,0.072
"(2, Corp HY)",0.033,0.109,0.128,2.979,0.106,0.155
"(3, EM Gov)",0.039,0.104,0.230,2.996,0.092,0.134
"(4, DM Equities)",0.038,0.193,0.127,3.217,0.216,0.306
"(5, EM Equities)",0.058,0.233,0.488,3.360,0.231,0.310
"(6, Private Equity)",0.103,0.255,0.492,3.525,0.206,0.304
"(7, Infrastructure)",0.071,0.180,0.442,3.366,0.147,0.216
"(8, Real Estate)",0.056,0.134,0.357,3.202,0.111,0.161
"(9, Hedge Funds)",0.052,0.102,0.071,3.188,0.080,0.125


In [8]:
# Verify CVaR view implementation (should be equal to 1.5)
stats_posterior['90%-CVaR'].iloc[4] / stats_prior['90%-CVaR'].iloc[4]

1.5

# Specify portfolios and constraints
Note that the current exposures are used for optimizations with transaction costs, while the benchmark is used for optimization with tracking error constraints.

In [9]:
# Cash long-only optimization without transaction costs
benchmark0_cash = ia.Exposure(
    instruments0_cash, [0.3, 0.15, 0.05, 0.05, 0.23, 0.02, 0.05, 0.05, 0.05, 0.05])
current_portfolio0_cash = ia.Exposure(
    instruments0_cash, [0.25, 0.20, 0.025, 0.025, 0.28, 0.02, 0.05, 0.05, 0.05, 0.05])

# Cash long-only optimization with transaction costs
benchmark_cash = ia.Exposure(
    instruments_cash, [0.3, 0.15, 0.05, 0.05, 0.23, 0.02, 0.05, 0.05, 0.05, 0.05])
current_portfolio_cash = ia.Exposure(
    instruments_cash, [0.25, 0.20, 0.025, 0.025, 0.28, 0.02, 0.05, 0.05, 0.05, 0.05])

# Derivatives optimization without transaction costs
benchmark0 = ia.Exposure(
    instruments0, [0.3, 0.15, 0.05, 0.05, 0.23, 0.02, 0.05, 0.05, 0.05, 0.05, 0, 0, 0, 0, 0, 0])
current_portfolio0 = ia.Exposure(
    instruments0, [0.25, 0.20, 0.025, 0.025, 0.28, 0.02, 0.05, 0.05, 0.05, 0.05, 0, 0, 0, 0, 0, 0])

# Derivatives optimization with transaction costs
benchmark = ia.Exposure(
    instruments, [0.3, 0.15, 0.05, 0.05, 0.23, 0.02, 0.05, 0.05, 0.05, 0.05, 0, 0, 0, 0, 0, 0])
current_portfolio = ia.Exposure(
    instruments, [0.25, 0.20, 0.025, 0.025, 0.28, 0.02, 0.05, 0.05, 0.05, 0.05, 0, 0, 0, 0, 0, 0])

In [10]:
I_cash = len(benchmark_cash)
I = len(benchmark)

# Constraints cash-only portfolio without transaction costs
pf0_cash = ia.Portfolio(current_portfolio0_cash, benchmark0_cash)
bounds_lower_cash = np.zeros(I_cash)
bounds_upper_cash = np.ones(I_cash)
pf0_cash.set_individual_bounds(bounds_lower_cash, bounds_upper_cash)

# Constraints cash-only portfolio with transaction costs
pf_cash = ia.Portfolio(current_portfolio_cash, benchmark_cash)
bounds_lower_cash = np.zeros(I_cash)
bounds_upper_cash = np.ones(I_cash)
pf_cash.set_individual_bounds(bounds_lower_cash, bounds_upper_cash)

# Constraints derivatives portfolios without transaction costs
pf0 = ia.Portfolio(current_portfolio0, benchmark0)
bounds_lower = np.zeros(I)
bounds_lower[-6:] = -0.5  # Derivatives shorting bounds
bounds_upper = np.ones(I)
bounds_upper[-6:] = 0.5  # Derivatives upper bounds
pf0.set_individual_bounds(bounds_lower, bounds_upper)
pf0.set_gross_exposure_max(2)  # Total derivatives leverage at most 2x

# Constraints derivatives portfolios with transaction costs
pf = ia.Portfolio(current_portfolio, benchmark)
bounds_lower = np.zeros(I)
bounds_lower[-6:] = -0.5  # Derivatives shorting bounds
bounds_upper = np.ones(I)
bounds_upper[-6:] = 0.5  # Derivatives upper bounds
pf.set_individual_bounds(bounds_lower, bounds_upper)
pf.set_gross_exposure_max(2)  # Total derivatives leverage at most 2x

# Prior portfolio optimizations

In [11]:
# For standard CVaR optimization
ia.cvar_options['demean_cvar'] = False

# Define optimization objects with different instruments and with/without transaction costs
opt0_cash = ia.Optimization(pf0_cash, pnl_cash, p)
opt_cash = ia.Optimization(pf_cash, pnl_cash, p)
opt0 = ia.Optimization(pf0, pnl, p)
opt = ia.Optimization(pf, pnl, p)

# Number of portfolios for resampled optimization
B = 100

In [13]:
# Prior cash problem without transaction costs
start_time = time()
frontier0_cash = opt0_cash.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR long-only cash efficient frontiers with 9 portfolios computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res0.npy', frontier0_cash.values)
np.save('means/means0', opt0_cash.means_bootstrap)

start_time = time()
portfolio0_cash = opt0_cash.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash target CVaR risk efficient portfolios computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res1.npy', portfolio0_cash.values)
np.save('means/means1', opt0_cash.means_bootstrap)

start_time = time()
portfolio_benchmark_opt0_cash = opt0_cash.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash CVaR risk budget efficient portfolios computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res2.npy', portfolio_benchmark_opt0_cash.values)
np.save('means/means2', opt0_cash.means_bootstrap)

100 CVaR long-only cash efficient frontiers with 9 portfolios computed in 7.3 seconds.

100 long-only cash target CVaR risk efficient portfolios computed in 4.63 seconds.

100 long-only cash CVaR risk budget efficient portfolios computed in 6.72 seconds.


In [14]:
# Prior cash problem with transaction costs
start_time = time()
frontier_cash = opt_cash.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR long-only cash efficient frontiers with 9 portfolios ' + 
      f'and transaction costs computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res3.npy', frontier_cash.values)
np.save('means/means3', opt_cash.means_bootstrap)

start_time = time()
portfolio_cash = opt_cash.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash target CVaR risk efficient portfolios ' + 
      f'with transaction costs computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res4.npy', portfolio_cash.values)
np.save('means/means4', opt_cash.means_bootstrap)

start_time = time()
portfolio_benchmark_opt_cash = opt_cash.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash CVaR risk budget efficient portfolios ' + 
      f'with transaction costs computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res5.npy', portfolio_benchmark_opt_cash.values)
np.save('means/means5', opt_cash.means_bootstrap)

100 CVaR long-only cash efficient frontiers with 9 portfolios and transaction costs computed in 8.54 seconds.

100 long-only cash target CVaR risk efficient portfolios with transaction costs computed in 4.73 seconds.

100 long-only cash CVaR risk budget efficient portfolios with transaction costs computed in 6.93 seconds.


In [15]:
# Full prior problem without transaction costs
start_time = time()
frontier0 = opt0.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR efficient frontiers with 9 portfolios, derivatives\n' +
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res6.npy', frontier0.values)
np.save('means/means6', opt0.means_bootstrap[:, :I_cash])

start_time = time()
portfolio0 = opt0.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} target CVaR risk efficient portfolios with derivatives\n'
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res7.npy', portfolio0.values)
np.save('means/means7', opt0.means_bootstrap[:, :I_cash])

start_time = time()
portfolio_benchmark_opt0 = opt0.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR risk budget efficient portfolios with derivatives\n'
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res8.npy', portfolio_benchmark_opt0.values)
np.save('means/means8', opt0.means_bootstrap[:, :I_cash])

100 CVaR efficient frontiers with 9 portfolios, derivatives
and absolute leverage constraint computed in 16.53 seconds.

100 target CVaR risk efficient portfolios with derivatives
and absolute leverage constraint computed in 5.4 seconds.

100 CVaR risk budget efficient portfolios with derivatives
and absolute leverage constraint computed in 9.11 seconds.


In [16]:
# Full prior problem
start_time = time()
frontier = opt.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR efficient frontiers with 9 portfolios, derivatives, transaction\n' +
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res9.npy', frontier.values)
np.save('means/means9', opt.means_bootstrap[:, :I_cash])

start_time = time()
portfolio = opt.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} target CVaR risk efficient portfolios with derivatives, transaction\n' +
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res10.npy', portfolio.values)
np.save('means/means10', opt.means_bootstrap[:, :I_cash])

start_time = time()
portfolio_benchmark_opt = opt.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR risk budget efficient portfolios with derivatives, transaction\n' +
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res11.npy', portfolio_benchmark_opt.values)
np.save('means/means11', opt.means_bootstrap[:, :I_cash])

100 CVaR efficient frontiers with 9 portfolios, derivatives, transaction
costs, and absolute leverage constraint computed in 20.29 seconds.

100 target CVaR risk efficient portfolios with derivatives, transaction
costs, and absolute leverage constraint computed in 5.7 seconds.

100 CVaR risk budget efficient portfolios with derivatives, transaction
costs, and absolute leverage constraint computed in 9.45 seconds.


# Posterior portfolio optimizations

In [17]:
# Define optimization objects with different instruments and with/without transaction costs
opt0_cash_q = ia.Optimization(pf0_cash, pnl_cash, q)
opt_cash_q = ia.Optimization(pf_cash, pnl_cash, q)
opt0_q = ia.Optimization(pf0, pnl, q)
opt_q = ia.Optimization(pf, pnl, q)

In [18]:
# Posterior cash problem without transaction costs
start_time = time()
frontier0_cash_q = opt0_cash_q.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR long-only cash efficient frontiers with 9 portfolios computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res12.npy', frontier0_cash_q.values)
np.save('means/means12', opt0_cash_q.means_bootstrap)

start_time = time()
portfolio0_cash_q = opt0_cash_q.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash target CVaR risk efficient portfolios computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res13.npy', portfolio0_cash_q.values)
np.save('means/means13', opt0_cash_q.means_bootstrap)

start_time = time()
portfolio_benchmark_opt0_cash_q = opt0_cash_q.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash CVaR risk budget efficient portfolios computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res14.npy', portfolio_benchmark_opt0_cash_q.values)
np.save('means/means14', opt0_cash_q.means_bootstrap)

100 CVaR long-only cash efficient frontiers with 9 portfolios computed in 6.29 seconds.

100 long-only cash target CVaR risk efficient portfolios computed in 4.43 seconds.

100 long-only cash CVaR risk budget efficient portfolios computed in 6.16 seconds.


In [19]:
# Posterior cash problem with transaction costs
start_time = time()
frontier_cash_q = opt_cash_q.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR long-only cash efficient frontiers with 9 portfolios ' + 
      f'and transaction costs computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res15.npy', frontier_cash_q.values)
np.save('means/means15', opt_cash_q.means_bootstrap)

start_time = time()
portfolio_cash_q = opt_cash_q.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash target CVaR risk efficient portfolios ' + 
      f'with transaction costs computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res16.npy', portfolio_cash_q.values)
np.save('means/means16', opt_cash_q.means_bootstrap)

start_time = time()
portfolio_benchmark_opt_cash_q = opt_cash_q.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} long-only cash CVaR risk budget efficient portfolios ' + 
      f'with transaction costs computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res17.npy', portfolio_benchmark_opt_cash_q.values)
np.save('means/means17', opt_cash_q.means_bootstrap)

100 CVaR long-only cash efficient frontiers with 9 portfolios and transaction costs computed in 6.89 seconds.

100 long-only cash target CVaR risk efficient portfolios with transaction costs computed in 4.58 seconds.

100 long-only cash CVaR risk budget efficient portfolios with transaction costs computed in 6.34 seconds.


In [20]:
# Full posterior problem without transaction costs
start_time = time()
frontier0_q = opt0_q.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR efficient frontiers with 9 portfolios, derivatives,\n' + 
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res18.npy', frontier0_q.values)
np.save('means/means18', opt0_q.means_bootstrap[:, :I_cash])

start_time = time()
portfolio0_q = opt0_q.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} target CVaR risk efficient portfolios with derivatives\n' + 
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res19.npy', portfolio0_q.values)
np.save('means/means19', opt0_q.means_bootstrap[:, :I_cash])

start_time = time()
portfolio_benchmark_opt0_q = opt0_q.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR risk budget efficient portfolios with derivatives\n' + 
      f'and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res20.npy', portfolio_benchmark_opt0_q.values)
np.save('means/means20', opt0_q.means_bootstrap[:, :I_cash])

100 CVaR efficient frontiers with 9 portfolios, derivatives,
and absolute leverage constraint computed in 19.24 seconds.

100 target CVaR risk efficient portfolios with derivatives
and absolute leverage constraint computed in 5.49 seconds.

100 CVaR risk budget efficient portfolios with derivatives
and absolute leverage constraint computed in 9.73 seconds.


In [21]:
# Full posterior problem
start_time = time()
frontier_q = opt_q.efficient_frontier(
    num_portfolios=9, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR efficient frontiers with 9 portfolios, derivatives, transaction\n' +
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res21.npy', frontier_q.values)
np.save('means/means21', opt_q.means_bootstrap[:, :I_cash])

start_time = time()
portfolio_q = opt_q.efficient_portfolio(
    target_risk=0.10, risk='cvar', num_samples=B, cvar_alpha=0.9,
    stacking='average', keep_bootstrap_results=True)
print(f'{B} target CVaR risk efficient portfolios with derivatives, transaction\n' + 
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.\n')
np.save('results/res22.npy', portfolio_q.values)
np.save('means/means22', opt_q.means_bootstrap[:, :I_cash])

start_time = time()
portfolio_benchmark_opt_q = opt_q.benchmark_efficient_portfolio(
    target_risk=0.10, tracking_error_max=0.04, risk='cvar', num_samples=B,
    cvar_alpha=0.9, stacking='average', keep_bootstrap_results=True)
print(f'{B} CVaR risk budget efficient portfolios with derivatives, transaction\n' +
      f'costs, and absolute leverage constraint computed in {np.round(time() - start_time, 2)} seconds.')
np.save('results/res23.npy', portfolio_benchmark_opt_q.values)
np.save('means/means23', opt_q.means_bootstrap[:, :I_cash])

100 CVaR efficient frontiers with 9 portfolios, derivatives, transaction
costs, and absolute leverage constraint computed in 24.42 seconds.

100 target CVaR risk efficient portfolios with derivatives, transaction
costs, and absolute leverage constraint computed in 5.63 seconds.

100 CVaR risk budget efficient portfolios with derivatives, transaction
costs, and absolute leverage constraint computed in 10.63 seconds.


# Disclaimer
The material in this notebook has been prepared by Fortitudo Technologies ApS and is general background information about the Investment Analysis module. This information is given in summary form and does not claim to be complete. The information should not be considered advice.

Fortitudo Technologies ApS can under no circumstances be held liable for any indirect, incidental, or consequential damages arising from the information contained in this notebook.

© 2021-2025 Fortitudo Technologies ApS. All rights reserved.